In [2]:
import os
import struct
import numpy as np
from matplotlib import pyplot
import matplotlib as mpl


def read_idx(filename):
    with open(filename, 'rb') as f:
        zero, data_type, dims = struct.unpack('>HBB', f.read(4))
        shape = tuple(struct.unpack('>I', f.read(4))[0] for d in range(dims))
        return np.fromstring(f.read(), dtype=np.uint8).reshape(shape)

def show(image):
    """
    Render a given numpy.uint8 2D array of pixel data.
    """

    fig = pyplot.figure()
    ax = fig.add_subplot(1,1,1)
    imgplot = ax.imshow(image, cmap=mpl.cm.Greys)
    imgplot.set_interpolation('nearest')
    ax.xaxis.set_ticks_position('top')
    ax.yaxis.set_ticks_position('left')
    pyplot.show()

In [3]:
train_img = read_idx('../../Downloads/Kek/train-images.idx3-ubyte')
train_lb = read_idx('../../Downloads/Kek/train-labels.idx1-ubyte')

test_img = read_idx('../../Downloads/Kek/t10k-images.idx3-ubyte')
test_lb = read_idx('../../Downloads/Kek/t10k-labels.idx1-ubyte')

proc_train_img = train_img.reshape(train_img.shape[0], train_img.shape[1] * train_img.shape[2])
proc_test_img = test_img.reshape(test_img.shape[0], test_img.shape[1] * test_img.shape[2])

C:\Users\liudm\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  if sys.path[0] == '':


In [15]:
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [7]:
logreg = LogisticRegression(n_jobs=4)
logreg.fit(proc_train_img[:5000], train_lb[:5000])

C:\Users\liudm\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 4.
  " = {}.".format(self.n_jobs))


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=4,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [26]:
np.sum(logreg.predict(proc_test_img) == test_lb) / len(test_lb)

0.8283

In [25]:
res = []
for C in np.power(10., np.arange(-5, 5)):
    svmclass = SVC(C=C)
    svmclass.fit(proc_train_img[:1000], train_lb[:1000])
    res.append(np.sum(svmclass.predict(proc_train_img[10000:11000]) == train_lb[10000:11000]) / len(train_lb[10000:11000]))
    print(C, res[-1])

1e-05 0.097
0.0001 0.097
0.001 0.097
0.01 0.097
0.1 0.097
1.0 0.097
10.0 0.097
100.0 0.097
1000.0 0.097
10000.0 0.097


In [21]:
np.sum(svmclass.predict(proc_test_img[:1000]) == test_lb[:1000]) / len(test_lb[:1000])

0.126

1
